# Convergence tests

## Linear Advection with Constant Coefficient

In [132]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


#Local libraries
from basis_nodes import generate_lagrange_poly, generate_LGL_points
from SBP_matrices import first_order_D, first_order_P_Q
from RHS_1D_advection import generate_RHS_1D_advection
from SSPRK import SSPRK33

### Defining problem and exact solution to test convergence

In [133]:
#Define physical domain
xlower = 0.
xupper = 2.

#Advection speed
c_advection = 1.

#Define initial conditions for convergence test
initial_c = lambda x: np.exp(-50*(x-1)**2)+1

#Define exact solution (Periodic boundary conditions)

def periodic_function(func, period, offset):
    return lambda x: func( ((x - offset) % period ) + offset )
def exact_sol(x,t):
    toeval = periodic_function(initial_c,xupper-xlower,0.)
    return toeval(x-c_advection*t)

### Different combinations of DOFs

In [134]:
#Spatial refinement
h_refinement = np.array([5,10,20,40,80]) #Number of elements
p_refinement = np.array([3,4,5,6,7]) #Number of nodes per element

ref_level_h = len(h_refinement)
ref_level_p = len(p_refinement)

In [135]:
#Useful function
min_array_not_zero = lambda array: np.min(array[array>1.e-16])

#Computing errors
tfinal = 1.
CFL=0.8
error_list = [[] for i in range(ref_level_h)]
for idx_h in range(ref_level_h):
    #h refinement
    h = h_refinement[idx_h]
    for idx_p in range(ref_level_p):
        #p refinement
        p = p_refinement[idx_p]

        #Get RHS
        RHS_local, xgrid_local, xiLGL_local = generate_RHS_1D_advection(xlower=xlower,
        xupper=xupper, n_elements=h,order_x=p,
        c_advection=c_advection,BC="periodic",num_flux_type="upwind")

        #Compute time step
        dx = np.min(np.diff(xgrid_local[:p]))
        dxi = np.min(np.diff(xiLGL_local))
        dt = CFL*dx*dxi/c_advection/2

        #Compute numerical solution
        u0 = initial_c(xgrid_local)
        u_frames, t_eval = SSPRK33(fun=RHS_local, u0=u0, dt=dt, t0=0.,tfinal=tfinal, nframes=2)

        #Compute error
        u_numerical = u_frames[-1]
        #Compute exact solution
        u_exact = exact_sol(xgrid_local,t_eval[-1]) 
        #Compute erro
        error = np.linalg.norm(u_numerical-u_exact,ord=2)*np.sqrt(dx)
        error_list[idx_h].append(error)

/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/basis_nodes.py:21: RuntimeWarning: invalid value encountered in divide
  return np.prod((x-x_nodes)/(xj-x_nodes), where = mask)
/Users/munozmca/Desktop/Courses/Spring_2023/SBP_Operators_AMCS348/basis_nodes.py:21: RuntimeWarning: divide by zero encountered in divide
  return np.prod((x-x_nodes)/(xj-x_nodes), where = mask)


In [136]:
#Displaying results
df = pd.DataFrame(error_list)
df.columns = p_refinement
df.index = h_refinement
#Adding the proper headers
df.columns.name = "p"
df.index.name = "N"
print("Errors")
df

Errors


p,3,4,5,6,7
N,,,,,
5,0.234724,0.126404,0.056383,0.024254,0.010903
10,0.126570,0.047250,0.015564,0.006071,0.002931
20,0.069584,0.019312,0.006585,0.002757,0.001322
40,0.036333,0.009573,0.003298,0.001380,0.000661
80,0.018262,0.004783,0.001649,0.000690,0.000331


In [137]:
#Errors for p=3
error_p3 = pd.DataFrame(df[7])
#Putting a title on error_p3
error_p3.name = "p=3"

#Adding a column to error_p3
col_values = np.log(error_p3.iloc[1:,0].values/error_p3.iloc[:-1,0].values)/np.log(1/2)
col_values = np.append(np.nan,col_values)
error_p3["rate"] = col_values
error_p3

,7,rate
N,,
5,0.010903,NaN
10,0.002931,1.895219
20,0.001322,1.148164
40,0.000661,0.999528
80,0.000331,1.000000
